**<center><h1>Tools for Customer Outreach Campaigns Using CrewAI Multi-Agents</h1></center>**

At the outset of this notebook, we explore the functionality of CrewAI's multi-agent system as a powerful tool for customer outreach. The system utilizes two primary agents, each with distinct roles and objectives to streamline lead generation and engagement.

1. **Sales Representative Agent**: This agent is responsible for identifying high-value leads that fit CrewAI’s ideal customer profile. With a keen eye for trends and opportunities, the Sales Representative agent leverages data analysis to generate targeted leads. This agent’s work forms the foundational stage of the campaign, ensuring that efforts are focused on the most promising prospects.

2. **Lead Sales Representative Agent**: Once leads are identified, the Lead Sales Representative takes over to foster meaningful relationships through tailored outreach. With a focus on personalized, compelling communications, this agent crafts messages that align with the prospect’s business objectives and recent milestones, making the connection between CrewAI’s offerings and the lead’s unique needs.

The tasks below outline the steps in this outreach journey:

- **Lead Profiling Task**: Conducted by the Sales Representative, this task involves creating a detailed profile of each lead, covering their background, key personnel, recent developments, and areas where CrewAI's solutions could add value.

- **Personalized Outreach Task**: Building on the profiling insights, the Lead Sales Representative crafts custom communication for decision-makers, addressing specific milestones and needs with a tone and strategy aligned with each lead's corporate culture.

Together, these agents and tasks optimize the outreach process, ensuring precision in targeting and personalization, which are critical for engaging and converting high-value leads.

## **Step 1. Install Libraries**

In [1]:
%%capture
! pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

## **Step 2. Import Libraries**

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In this notebook, we are going to use gpt-4o-mini by OpenAI which is the default llm used in CrewAI. Also, we are going to use Serper API which is going to allow us to use google for search.

In [3]:
from crewai import Agent, Task, Crew
import os

os.environ["OPENAI_API_KEY"] = "*****************************************************"
os.environ["SERPER_API_KEY"] = "*****************************************************"

## **Step 3. Define Agents**

In [6]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [7]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

## **Step 4. Define Tools**

In [8]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [9]:
directory_read_tool = DirectoryReadTool(directory='./instructions') # Facilitates reading and processing of directory structures and their contents.
file_read_tool = FileReadTool() # Enables reading and extracting data from files, supporting various file formats.
search_tool = SerperDevTool()

To create a custom tool using CrewAI's `BaseTool` class, start by importing it from `crewai_tools`. Each tool requires a unique name and a descriptive explanation of its purpose. For demonstration purposes, the `SentimentAnalysisTool` here is designed to return a positive sentiment for any text input, making it simple and effective for classroom scenarios. When running the tool locally, you can customize its behavior by adding your own logic to the `_run` function, tailoring it to meet specific analysis requirements.

In [10]:
from crewai_tools import BaseTool

In [11]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [12]:
sentiment_analysis_tool = SentimentAnalysisTool()

## **Step 5. Define Tasks**

In [13]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

In [14]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## **Step 6. Creating the Crew**

In [15]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
	memory=True
)

## **Step 7. Running the Crew**

In [16]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative

 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.

Don't make assumptions and only use information you absolutely sure about.





> Entering new CrewAgentExecutor chain...

The first step to conduct an in-depth analysis of the company DeepLearningAI is to gather as much information as possible. I should start by searching the internet to get a broad overview of the company.



Action: 

Search the internet



Action Input: 

{"search_query": "DeepLearningAI company profile"}

 





Search results: Title: Deeplearning.ai - Crunchbase Company Profile & Funding

Link: ht

In [17]:
from IPython.display import Markdown
Markdown(result)

Draft 1: 
"Dear Andrew Ng,

As a leader in AI education, you've made remarkable strides with DeepLearningAI's mission to make world-class AI education accessible to everyone. We at CrewAI recognize and appreciate your efforts. Our solutions, designed to facilitate effective communication and interpret machine learning models, can support your ongoing innovations. Imagine enhancing your online courses with interactive AI tools, making complex concepts even more accessible to your learners. We look forward to the opportunity to collaborate in fostering a more AI-literate world.

Best regards,
[Your Name]"

Draft 2: 
"Dear Andrew Ng,

Congratulations on the recent launch of DeepLearningAI's certificate in Generative AI for Software Development. This is a testament to your commitment to innovation and expanding educational offerings in the AI field. At CrewAI, we share a similar passion for AI and its potential to transform the way we learn and communicate. Our solutions could highly complement your courses by providing interactive learning experiences and making AI concepts more understandable for non-technical users. We are excited about the possibility of collaborating with you to further advance AI education across the globe.

Best regards,
[Your Name]"